# Experiment_dataset

In [2]:
import pandas as pd
from dataset.data_manager import BuildDataset

from dataset.dataset_labeling_study import aggiorna_label_distanza_temporale
from dataset.dataset_labeling_study import create_tile_videos_relabeling, create_df_video_from_master_df_relabeling

In [2]:
# NON carico quello con tutti i metodi all_data_all_methods_tracks_complete_fast.csv

sup_data_train = BuildDataset(type='SUPERVISED', master_df_path="all_data_CL7_tracks_complete_fast.csv")
sup_data_train.load_master_df()

In [3]:
sup_data_train.master_df.dtypes

path             string[python]
datetime         datetime64[ns]
tile_offset_x             int16
tile_offset_y             int16
x_pix                    object
y_pix                    object
source           string[python]
id_cyc_unico              int32
start_time       datetime64[ns]
end_time         datetime64[ns]
label                      int8
dtype: object

In [4]:
sup_data_train.calc_delta_time()

In [5]:
sup_data_train.master_df

,path,datetime,tile_offset_x,tile_offset_y,x_pix,y_pix,source,id_cyc_unico,start_time,end_time,label,delta_time
0,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,0,0,[],[],[],0,NaT,NaT,0,NaT
1,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,213,0,[],[],[],0,NaT,NaT,0,NaT
2,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,426,0,[],[],[],0,NaT,NaT,0,NaT
3,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,639,0,[],[],[],0,NaT,NaT,0,NaT
4,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,852,0,[],[],[],0,NaT,NaT,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
1413163,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,213,196,[],[],[],0,NaT,NaT,0,NaT
1413164,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,426,196,[],[],[],0,NaT,NaT,0,NaT
1413165,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,639,196,[],[],[],0,NaT,NaT,0,NaT
1413166,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,852,196,[],[],[],0,NaT,NaT,0,NaT


In [6]:
new_label = aggiorna_label_distanza_temporale(sup_data_train.master_df, soglia=pd.Timedelta(hours=12), sub_lab=-1)
new_label

Colonna label_12_00 aggiunta


'label_12_00'

In [ ]:
######################################
# # verifico che il relabeling sia corretto per la parte iniziale e finale della durata del ciclone
pd.set_option('display.max_rows', 500)
m = sup_data_train.master_df[new_label] == -1
moff = (sup_data_train.master_df[['tile_offset_x','tile_offset_y']] == (213,0)).all(axis=1)

m_cyc = sup_data_train.master_df.id_cyc_unico == 7001328
sup_data_train.master_df[m_cyc].loc[20545:23700] #14545:14600]

### ora levo le righe con label == -1 e poi prendo la colonna relabel come label ufficiale

In [7]:
m = sup_data_train.master_df[new_label] == -1
df_mod = sup_data_train.master_df[~m].copy().drop(columns='label').rename(columns={new_label:'label'})

In [8]:
df_mod

,path,datetime,tile_offset_x,tile_offset_y,x_pix,y_pix,source,id_cyc_unico,start_time,end_time,delta_time,label
0,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,0,0,[],[],[],0,NaT,NaT,NaT,0
1,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,213,0,[],[],[],0,NaT,NaT,NaT,0
2,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,426,0,[],[],[],0,NaT,NaT,NaT,0
3,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,639,0,[],[],[],0,NaT,NaT,NaT,0
4,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,852,0,[],[],[],0,NaT,NaT,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1413163,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,213,196,[],[],[],0,NaT,NaT,NaT,0
1413164,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,426,196,[],[],[],0,NaT,NaT,NaT,0
1413165,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,639,196,[],[],[],0,NaT,NaT,NaT,0
1413166,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,852,196,[],[],[],0,NaT,NaT,NaT,0


In [9]:
sup_data_train.make_df_video(new_master_df=df_mod, output_dir=None,  is_to_balance=True) #, idxs=[1,2,3,4,5,6,7,8])

1)  ->
Num video CON cicloni: 104, Num video SENZA cicloni: 2018	Totale video tiles: 2122		Bilanciamento video...
 video senza cicloni tenuti: 104
208 video per il periodo (effettivo) da 2011-11-01 00:00:00 a 2011-11-10 22:10:00

2)  ->
Num video CON cicloni: 99, Num video SENZA cicloni: 2016	Totale video tiles: 2115		Bilanciamento video...
 video senza cicloni tenuti: 99
198 video per il periodo (effettivo) da 2014-11-03 01:20:00 a 2014-11-12 23:55:00

3)  ->
Num video CON cicloni: 88, Num video SENZA cicloni: 1622	Totale video tiles: 1710		Bilanciamento video...
 video senza cicloni tenuti: 88
176 video per il periodo (effettivo) da 2016-10-26 00:00:00 a 2016-11-02 23:55:00

4)  ->
Num video CON cicloni: 83, Num video SENZA cicloni: 1618	Totale video tiles: 1701		Bilanciamento video...
 video senza cicloni tenuti: 83
166 video per il periodo (effettivo) da 2017-11-13 00:00:00 a 2017-11-20 15:55:00

5)  ->
Num video CON cicloni: 61, Num video SENZA cicloni: 998	Totale video tiles: 105

In [10]:
df_v = sup_data_train.df_video.copy()

In [11]:
print(df_v.label.sum(), df_v.shape[0])

1773 3546


In [17]:
train_p = 0.7
len_p = int(train_p*df_v.shape[0])
df_video_train = df_v.sort_values('start_time').iloc[:len_p]
df_video_test = df_v.sort_values('start_time').iloc[len_p:]
df_video_train.shape, df_video_test.shape

((2482, 8), (1064, 8))

# Split train e test dal file di Manos

### costruisco il dataset dei cicloni 10

In [ ]:
from dataset.data_manager import BuildDataset


import pandas as pd
from intervaltree import IntervalTree

output_dir = "../airmassRGB/supervised/" 
input_dir="../fromgcloud"

In [18]:
tracks_df_MED_CL10 = pd.read_csv("manos_CL10_pixel.csv", parse_dates=['time', 'start_time', 'end_time'])
tracks_df_MED_CL10.shape[0]

3397

In [ ]:
# divido le track di Manos in train e test
cicloni_unici = tracks_df_MED_CL10.id_cyc_unico.unique()
train_p = 0.7
len_p = int(train_p*cicloni_unici.shape[0])
cicloni_unici_train = cicloni_unici[:len_p]
cicloni_unici_test = cicloni_unici[len_p:]
cicloni_unici_train.shape, cicloni_unici_test.shape


((28,), (12,))

In [20]:
tracks_df_train = tracks_df_MED_CL10[tracks_df_MED_CL10.id_cyc_unico.isin(cicloni_unici_train)]
tracks_df_test = tracks_df_MED_CL10[tracks_df_MED_CL10.id_cyc_unico.isin(cicloni_unici_test)]

In [21]:
train_m = BuildDataset(type='SUPERVISED')
train_m.get_data_ready(tracks_df_train, input_dir, output_dir)

test_m = BuildDataset(type='SUPERVISED')
test_m.get_data_ready(tracks_df_test, input_dir, output_dir)

117764 files loaded.
Colonna label_12_00 aggiunta
1)  ->
Num video CON cicloni: 0, Num video SENZA cicloni: 2160	Totale video tiles: 2160		Bilanciamento video...
 video senza cicloni tenuti: 0
No video present for period: 2011-11-01 00:00:00 to 2011-11-11 00:00:00
2)  ->
Num video CON cicloni: 74, Num video SENZA cicloni: 2067	Totale video tiles: 2141		Bilanciamento video...
 video senza cicloni tenuti: 74
Creazione delle folder per i 74 video...	../airmassRGB/supervised/06-11-2014_0235_213_0
../airmassRGB/supervised/11-11-2014_0115_639_196
../airmassRGB/supervised/11-11-2014_1035_0_196
../airmassRGB/supervised/11-11-2014_2115_426_196
../airmassRGB/supervised/10-11-2014_1315_852_0
../airmassRGB/supervised/12-11-2014_0755_0_196
../airmassRGB/supervised/10-11-2014_1835_426_0
../airmassRGB/supervised/07-11-2014_0915_0_196
../airmassRGB/supervised/11-11-2014_0115_426_196
../airmassRGB/supervised/10-11-2014_1955_1065_196
../airmassRGB/supervised/10-11-2014_1835_213_196
../airmassRGB/supervi

KeyboardInterrupt: 